In [1]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt
import defmod.hamiltonian_multishape as hamiltonian
import model_MultiShapeCirclesTranslation as model

torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
nb_pts1 = 10
nb_pts2 = 7
dim = 2

o1 = torch.tensor([0.,0.]).float()
o2 = torch.tensor([10.,0.]).float()
origin = [o1,o2]
r1 = 2
r2 = 1
radius = [r1, r2]
nb_pts = [nb_pts1, nb_pts2]
source = mm.multipleCircles(origin, radius, nb_pts)

origin_target = [[10., 0.], [0.,0.]]
target = mm.multipleCircles(origin_target, radius, nb_pts)

In [3]:
sigma1 = 20
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=source[0].view(-1))
trans1 = dm.deformationmodules.GlobalTranslation(manifold1, sigma1)

sigma2 = 20
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=source[1].view(-1))
trans2 = dm.deformationmodules.GlobalTranslation(manifold2, sigma2)

module_list = [trans1, trans2]
sigma_bg = 1
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)

In [4]:
constr_Id = dm.constraints.Identity()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

#gd0_list = [*source, source]
gd0_list = [source[0].double(), source[1].double(), [source[0].double(), source[1].double()]]
mom0_list = [*[torch.zeros(gd.shape) for gd in modules.manifold.gd[:-1]], [torch.zeros(gd.shape) for gd in modules.manifold.gd[-1]]]

H.module.manifold.fill_gd([gd0_list[0].view(-1), gd0_list[1].view(-1), [gd0_list[2][0].view(-1), gd0_list[2][1].view(-1)]])
H.module.manifold.fill_cotan([mom0_list[0].view(-1), mom0_list[1].view(-1), [mom0_list[2][0].view(-1), mom0_list[2][1].view(-1)]])
H.geodesic_controls()

In [6]:
# 2 Modules
def controlslist2tensor(controls_list):
    return torch.cat([controls_list[0].view(-1),controls_list[1].view(-1), controls_list[-1][0].view(-1),controls_list[-1][1].view(-1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], controls[dim*nb_pts[0]:dim*(nb_pts[0]+nb_pts[1])], [controls[dim*n:dim*(n+nb_pts[0])], controls[dim*(n+nb_pts[0]):]]]

def gdtensor2list(gdtensor, nb_pts, dim):
    gdlist = []
    gdlist_bg = []
    j = 0
    n = sum(nb_pts)
    for i in range(len(nb_pts)):
        gdlist.append(gdtensor[dim*j:dim*(j+nb_pts[i])])
        gdlist_bg.append(gdtensor[dim*(n+j):dim*(n+j+nb_pts[i])])
        j = j + nb_pts[i]
    return [*gdlist, gdlist_bg]

def gdlist2tensor(gdlist):
    return torch.cat([*gdlist[:-1],*gdlist[-1]],0)

In [7]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, nb_pts, dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, nb_pts, dim))
        
        return H.module.autoaction()

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

test_gradcheck_autoaction()

True

In [8]:
def test_gradcheck_Hamiltonian_2modules():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        H.geodesic_controls()
                
        return H()
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

#modules.compute_geodesic_variables(modules.manifold, constr_Id)

gd = gdlist2tensor(modules.manifold.gd)
mom = controlslist2tensor(modules.manifold.cotan)


test_gradcheck_Hamiltonian_2modules()

True

In [9]:
def test_gradcheck_shooting():
    def shooting(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        intermediate_states, intermediate_controls = dm.shooting.shoot_euler(H, it=2)
        return [H.module.manifold.cotan[0], H.module.manifold.cotan[1], H.module.manifold.cotan[2][0], H.module.manifold.cotan[2][1]]
    return torch.autograd.gradcheck(shooting, (gd, mom), raise_exception=True,atol=1e-2)

gd = gdlist2tensor(modules.manifold.gd)
mom = controlslist2tensor(modules.manifold.cotan)


test_gradcheck_shooting()

KeyboardInterrupt: 